## Using a local GA4GH Data Connect server
This notebook was run against a DC service running against a local trino server



In [2]:
from fasp.search import DataConnectClient
cl = DataConnectClient('http://localhost:8089/')
#cl.listTables(verbose=True)

This lists the schema for one of the dbGaP asthma studies.

In [4]:
cl.listTableInfo('bigquery.dbgap_synth.TOPMed_WGS_CAMP_Subject_Phenotypes', verbose=True)

_Schema for tablebigquery.dbgap_synth.TOPMed_WGS_CAMP_Subject_Phenotypes_
{
   "name": "bigquery.dbgap_synth.TOPMed_WGS_CAMP_Subject_Phenotypes",
   "data_model": {
      "description": "This subject phenotypes data table includes subject's asthma status, gender, age, race, height, weight, BMI, and smoking status (n=3 variables; ever, current, and former).",
      "$id": "dbgap:pht009836.v2",
      "properties": {
         "gender": {
            "$id": "dbgap:phv00426560.v2",
            "description": "Gender of participant",
            "type": "encoded value",
            "oneOf": [
               {
                  "const": "1",
                  "title": "Male"
               },
               {
                  "const": "2",
                  "title": "Female"
               }
            ]
         },
         "race": {
            "$id": "dbgap:phv00426562.v2",
            "description": "Race of participant",
            "type": "string"
         },
         "SUBJECT_ID": {

Based on the above we can construct a query to answer a question we might be interested in.
This summarizes the cohort according to smoking status.

In [6]:
query1 = '''SELECT current_smoker, ever_smoker, count(*) n
FROM bigquery.dbgap_synth.TOPMed_WGS_CAMP_Subject_Phenotypes
group by current_smoker, ever_smoker
'''

res = cl.runQuery(query1, returnType='dataframe')
res

_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________
____Page7_______________
____Page8_______________
____Page9_______________


,current_smoker,ever_smoker,n
0,2,1,23
1,1,1,135
2,1,2,37
3,1,0,3
4,2,2,2


#### todo
show how the schema lets us decode those cryptic values

### Important point
A data user gets significant value out of the machine readable data dictionary.

### Another asthma dataset - WGS BAGS
See the dataset listing to learn this is the Barbados Asthma study

In [9]:
cl.listTableInfo('bigquery.dbgap_synth.TOPMed_WGS_BAGS_Subject_Phenotypes', verbose=True)

_Schema for tablebigquery.dbgap_synth.TOPMed_WGS_BAGS_Subject_Phenotypes_
{
   "name": "bigquery.dbgap_synth.TOPMed_WGS_BAGS_Subject_Phenotypes",
   "data_model": {
      "description": "This subject phenotype table includes age, sex, and affection status for asthma. Additional variables have been added: smoking status (n=9 variables; includes mother and father smoking status), asthma age of onset, allergic rhinitis, weight, height, IgE measurements (n=3 variables), phadiatop measurements, spirometry measurements (n=3 variables), bronchodilator pills/inhalers/nebulizer (n=6 variables), steroid pills/inhalers (n=4 variables), and cromolyn inhalers (n=2 variables).",
      "$id": "dbgap:pht005905.v3",
      "properties": {
         "AGE_START_SMOKING": {
            "$id": "dbgap:phv00375340.v1",
            "description": "Answer to \"At what age did you start smoking?\"",
            "type": "integer, encoded value",
            "$unit": "years",
            "oneOf": [
               {

In [10]:

query1 = '''SELECT AFFECTION_STATUS, FEV1_FVC, FVC_PREBD, RAW_PHADIATOP_PAU_L
FROM bigquery.dbgap_synth.TOPMed_WGS_BAGS_Subject_Phenotypes
where FVC_PREBD < 2.2
'''

res = cl.runQuery(query1, returnType='dataframe')
res

_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________
____Page7_______________
____Page8_______________


,AFFECTION_STATUS,FEV1_FVC,FVC_PREBD,RAW_PHADIATOP_PAU_L
0,2,None,1.920412,NA
1,2,None,2.165323,<0.1
2,0,None,2.152361,NA
3,1,None,2.105422,NA


### Simulacrum dataset

In [12]:
cl.listTableInfo('bigquery.simulacrum.av_patient', verbose=True)

_Schema for tablebigquery.simulacrum.av_patient_
{
   "name": "bigquery.simulacrum.av_patient",
   "description": "Patient table from Simulacrum data release, version 1.2.0-2017, 2021-01-19\n\nThis is a package of synthetic data based on cancer registration data from \nthe Cancer Outcomes and Services Dataset (COSD) and the Systemic Anti-Cancer \nTreatment (SACT) dataset collected by the National Cancer Registration and \nAnalysis Service (NCRAS).\n\nAny analysis performed on this synthetic data will not correspond exactly to \nanalysis performed on the real data. The synthetic data does not contain real \npatients but just mimics specific statistical properties of the real data. To \nminimise the potential for confusion with real data, organisational codes in \nthis extract have been obfuscated.\n\n\nFor more information on the Simulacrum and licensing details, please see\n  https://simulacrum.healthdatainsight.org.uk/",
   "data_model": {
      "description": "Patient table from Simu

In [50]:
query1 = '''SELECT *
FROM bigquery.simulacrum.av_patient
where DEATHCAUSECODE_1A = 'R54'
'''

res = cl.runQuery(query1, returnType='dataframe')
res

_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
____Page6_______________
____Page7_______________
____Page8_______________
____Page9_______________


,patientid,sex,linknumber,ethnicity,deathcausecode_1a,deathcausecode_1b,deathcausecode_1c,deathcausecode_2,deathcausecode_underlying,deathlocationcode,newvitalstatus,vitalstatusdate
0,10078592,2,810078592,A,R54,C182,"I10,I489",I269,,None,A,2018-11-11
1,10143453,1,810143453,A,R54,,,,I679,None,D,2015-11-23
2,10149196,1,810149196,None,R54,C349,,F109,C459,None,D,2017-09-13
3,60011606,2,860011606,A,R54,,,,C19,None,D,2016-08-13
4,60021681,1,860021681,A,R54,,,,C19,None,D,2018-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...
7856,290024421,1,1090024421,A,R54,,,I251,I259,X,A,2019-02-11
7857,300012197,1,1100012197,A,R54,,,,D432,X,A,2019-02-11
7858,320008204,2,1120008204,None,R54,K918,,,C64,X,A,2019-02-11
7859,320008900,2,1120008900,J,R54,,,,C66,X,A,2019-02-11


### Another asthma dataset

In [6]:
cl.listTableInfo('bigquery.dbgap_synth.TOPMed_WGS_CARE_PACT_Subject_Phenotypes', verbose=True)

_Schema for tablebigquery.dbgap_synth.TOPMed_WGS_CARE_PACT_Subject_Phenotypes_
{
   "name": "bigquery.dbgap_synth.TOPMed_WGS_CARE_PACT_Subject_Phenotypes",
   "data_model": {
      "description": "This subject phenotypes data table includes subject's age, sex, race, BMI, affection status for asthma, smoking status, hospital admission for asthma, prescribed oral steroids for asthma use, and delta FEV1 measurements.",
      "$id": "dbgap:pht010132.v1",
      "properties": {
         "SUBJECT_ID": {
            "$id": "dbgap:phv00430283.v1",
            "description": "De-identified Subject ID",
            "type": "string"
         },
         "SEX": {
            "$id": "dbgap:phv00430285.v1",
            "description": "Gender of participant",
            "type": "string"
         },
         "Race": {
            "$id": "dbgap:phv00430286.v1",
            "description": "Race of participant",
            "type": "string"
         },
         "Affection_Status": {
            "$id": "d

In [26]:
def findVariables(table, term):
    table_info = cl.listTableInfo(table)
    for var, prop in table_info.schema['data_model']['properties'].items():
        if term.lower() in var.lower():
            #print(var, prop['description'])

            print(var)

            
findVariables('bigquery.dbgap_synth.TOPMed_WGS_BAGS_Subject_Phenotypes', 'FEV')

FEV1_PREBD
FEV1_FVC


In [32]:
asthma_phenotype_tables = ['bigquery.dbgap_synth.TOPMed_WGS_CAMP_Subject_Phenotypes',
'bigquery.dbgap_synth.TOPMed_WGS_CARE_PACT_Subject_Phenotypes',
'bigquery.dbgap_synth.TOPMed_WGS_BAGS_Subject_Phenotypes',
'bigquery.copdgene.subject_phenotypes_hmb',
'bigquery.esp_lunggo_copdgene.subject_phenotypes_hmb',
'bigquery.dbgap_synth.camp_cdata',
'bigquery.dbgap_synth.camp_ldata',
'bigquery.mesa.airnrspirometry_hmb',
'bigquery.topmed_wgs_partners_afgen.subject_phenotypes_hmb']

for t in asthma_phenotype_tables:
    print(t)
    findVariables(t,'fev')
    print('_'*80)


bigquery.dbgap_synth.TOPMed_WGS_CAMP_Subject_Phenotypes
________________________________________________________________________________
bigquery.dbgap_synth.TOPMed_WGS_CARE_PACT_Subject_Phenotypes
________________________________________________________________________________
bigquery.dbgap_synth.TOPMed_WGS_BAGS_Subject_Phenotypes
FEV1_PREBD
FEV1_FVC
________________________________________________________________________________
bigquery.copdgene.subject_phenotypes_hmb
hayfev
hayfevage
hayfevagedk
hayfevdxbydr
hayfevstillhave
hayfevagestop
hayfevstill
hayfevtreat
fev1pp_post
fev1_fvc_post
fev1_post
fev6_post
fev1_pre
fev6_pre
fev1_fvc_pre
deltafev1
bdr_pct_fev1
________________________________________________________________________________
bigquery.esp_lunggo_copdgene.subject_phenotypes_hmb
________________________________________________________________________________
bigquery.dbgap_synth.camp_cdata
prefev_baseline
prefevpp_baseline
_______________________________________________

In [22]:
for t in asthma_phenotype_tables:
    print(t)
    cl.listTableColumns(t)
    print('_'*80)

bigquery.dbgap_synth.TOPMed_WGS_CAMP_Subject_Phenotypes
gender
_______________________________________
race
_______________________________________
SUBJECT_ID
_______________________________________
weight
_______________________________________
asthma
_______________________________________
current_smoker
_______________________________________
age
_______________________________________
former_smoker
_______________________________________
height
_______________________________________
bmi
_______________________________________
ever_smoker
_______________________________________
________________________________________________________________________________
bigquery.dbgap_synth.TOPMed_WGS_CARE_PACT_Subject_Phenotypes
SUBJECT_ID
_______________________________________
SEX
_______________________________________
Race
_______________________________________
Affection_Status
_______________________________________
asthma_hospital_Yes12mo
_______________________________________
age
____

In [16]:
cl.listTableInfo('bigquery.dbgap_synth.TOPMed_WGS_BAGS_Subject_Phenotypes', verbose=True)

_Schema for tablebigquery.dbgap_synth.TOPMed_WGS_BAGS_Subject_Phenotypes_
{
   "name": "bigquery.dbgap_synth.TOPMed_WGS_BAGS_Subject_Phenotypes",
   "description": "This subject phenotype table includes age, sex, and affection status for asthma. Additional variables have been added: smoking status (n=9 variables; includes mother and father smoking status), asthma age of onset, allergic rhinitis, weight, height, IgE measurements (n=3 variables), phadiatop measurements, spirometry measurements (n=3 variables), bronchodilator pills/inhalers/nebulizer (n=6 variables), steroid pills/inhalers (n=4 variables), and cromolyn inhalers (n=2 variables).",
   "data_model": {
      "$id": "dbgap:pht005905.v3",
      "description": "This subject phenotype table includes age, sex, and affection status for asthma. Additional variables have been added: smoking status (n=9 variables; includes mother and father smoking status), asthma age of onset, allergic rhinitis, weight, height, IgE measurements (n=3 

In [24]:
cl.listCatalog('bigquery')

Retrieving the table list
____Page1_______________
bigquery.1000genomes.1000genomes_drs
bigquery.1000genomes.aligned_read
bigquery.1000genomes.aliquot
bigquery.1000genomes.bdc_1000genomes
bigquery.1000genomes.demographics
bigquery.1000genomes.germline_variation_index
bigquery.1000genomes.phenotype_drs
bigquery.1000genomes.read_group
bigquery.1000genomes.sample
bigquery.1000genomes.simple_germline_variation
bigquery.1000genomes.ssd_drs
bigquery.1000genomes.ssd_drs_table
bigquery.1000genomes.subject
bigquery.100genomes.import2
bigquery.100genomes.import3
bigquery.100genomes.testimport
bigquery.blog_unnest.firebase_raw
bigquery.cda.cdav3
bigquery.cidr_gecco_colorectal_cancer_gxe.sample
bigquery.cidr_gecco_colorectal_cancer_gxe.sample_attributes
bigquery.cidr_gecco_colorectal_cancer_gxe.subject
bigquery.cidr_gecco_colorectal_cancer_gxe.subject_phenotypes
bigquery.cidr_ndd_g2.pedigree
bigquery.cidr_ndd_g2.sample_attributes
bigquery.cidr_ndd_g2.subject_phenotypes
bigquery.cineca.syn_africa_h

['bigquery.1000genomes.1000genomes_drs',
 'bigquery.1000genomes.aligned_read',
 'bigquery.1000genomes.aliquot',
 'bigquery.1000genomes.bdc_1000genomes',
 'bigquery.1000genomes.demographics',
 'bigquery.1000genomes.germline_variation_index',
 'bigquery.1000genomes.phenotype_drs',
 'bigquery.1000genomes.read_group',
 'bigquery.1000genomes.sample',
 'bigquery.1000genomes.simple_germline_variation',
 'bigquery.1000genomes.ssd_drs',
 'bigquery.1000genomes.ssd_drs_table',
 'bigquery.1000genomes.subject',
 'bigquery.100genomes.import2',
 'bigquery.100genomes.import3',
 'bigquery.100genomes.testimport',
 'bigquery.blog_unnest.firebase_raw',
 'bigquery.cda.cdav3',
 'bigquery.cidr_gecco_colorectal_cancer_gxe.sample',
 'bigquery.cidr_gecco_colorectal_cancer_gxe.sample_attributes',
 'bigquery.cidr_gecco_colorectal_cancer_gxe.subject',
 'bigquery.cidr_gecco_colorectal_cancer_gxe.subject_phenotypes',
 'bigquery.cidr_ndd_g2.pedigree',
 'bigquery.cidr_ndd_g2.sample_attributes',
 'bigquery.cidr_ndd_g2.